In [1]:
from torch_geometric.datasets import QM9
from torch_geometric.loader import DataLoader
import torch_geometric.transforms as T
from test_model import Net

In [2]:
# transform = T.Compose([MyTransform(), Complete(), T.Distance(norm=False)])

data = QM9(root='./practice_data', transform=None)

In [4]:
"""
each batch is considered a hugh graph with many nodes and edges,
in EGNN, they introduce the concept of l2 distance between nodes, 
yet I am not including this (probably not) for now. 


"""
dataloader = DataLoader(data, batch_size=32, shuffle=True) 
for i , x in enumerate(dataloader):
    print(x)
    if i > 20: 
        break
    


DataBatch(x=[580, 11], edge_index=[2, 1210], edge_attr=[1210, 4], y=[32, 19], pos=[580, 3], idx=[32], name=[32], z=[580], batch=[580], ptr=[33])
DataBatch(x=[581, 11], edge_index=[2, 1192], edge_attr=[1192, 4], y=[32, 19], pos=[581, 3], idx=[32], name=[32], z=[581], batch=[581], ptr=[33])
DataBatch(x=[560, 11], edge_index=[2, 1152], edge_attr=[1152, 4], y=[32, 19], pos=[560, 3], idx=[32], name=[32], z=[560], batch=[560], ptr=[33])
DataBatch(x=[589, 11], edge_index=[2, 1220], edge_attr=[1220, 4], y=[32, 19], pos=[589, 3], idx=[32], name=[32], z=[589], batch=[589], ptr=[33])
DataBatch(x=[596, 11], edge_index=[2, 1236], edge_attr=[1236, 4], y=[32, 19], pos=[596, 3], idx=[32], name=[32], z=[596], batch=[596], ptr=[33])
DataBatch(x=[587, 11], edge_index=[2, 1208], edge_attr=[1208, 4], y=[32, 19], pos=[587, 3], idx=[32], name=[32], z=[587], batch=[587], ptr=[33])
DataBatch(x=[566, 11], edge_index=[2, 1164], edge_attr=[1164, 4], y=[32, 19], pos=[566, 3], idx=[32], name=[32], z=[566], batch=[5

In [5]:
dummy = next(iter(dataloader))

In [6]:
print(dummy)
print(dummy.edge_index)

DataBatch(x=[564, 11], edge_index=[2, 1136], edge_attr=[1136, 4], y=[32, 19], pos=[564, 3], idx=[32], name=[32], z=[564], batch=[564], ptr=[33])
tensor([[  0,   0,   0,  ..., 561, 562, 563],
        [  1,   8,   9,  ..., 554, 555, 556]])


In [ ]:
from tqdm import tqdm 
# try the customized pyg model 
hidden_dim = 64 # hidden dimension
n_feat_out = 7 # output latent embedding shape
n_layers = 4
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
num_epochs = 10 
net = Net(n_feat_in=dummy.x.shape[1], hidden_dim=hidden_dim, n_feat_out= n_feat_out, device=device)
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)



for epoch in range(num_epochs):
    with tqdm(dataloader) as tepoch:
        for data in tepoch:
            optimizer.zero_grad()
            data = data.to(device)
            

            










# Node embedding 

Embedding the graph representation of the data. ()

In [11]:
import torch 
from torch import nn 
# num_embeddings = the number of unique items that can be embed, types of atoms 
# embedding_dim = embedding dimension, the size of the embedding vector (latent space)

embedding = nn.Embedding(num_embeddings = 10, embedding_dim = 3) 
# a batch of 2 samples of 4 indices each
input = torch.LongTensor([[1, 2, 4, 5], [4, 3, 2, 9]])
print(input.shape)
e = embedding(input)
e.shape

torch.Size([2, 4])


torch.Size([2, 4, 3])